In [83]:
using Pkg
Pkg.activate("Dev")
Pkg.add("FiniteDiff")
using FiniteDiff
using BenchmarkTools

  Activating project at `~/repos/Skylight.jl`


   Resolving package versions...


  No Changes to `~/repos/Skylight.jl/Project.toml`


  No Changes to `~/repos/Skylight.jl/Manifest.toml`


   Resolving package versions...


...

Precompiling project...


  ✓ Parsers


  ✓ JSON


  ✓ BenchmarkTools


  ✓ Skylight
  4 dependencies successfully precompiled in 47 seconds. 163 already precompiled.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version


In [115]:
using Skylight

function finite_difference_metric_jacobian(position, spacetime::AbstractSpacetime, cache)
    return FiniteDiff.finite_difference_jacobian(Skylight.metric_field(spacetime), position, cache)
end

function finite_difference_metric_jacobian!(∂g, position, spacetime::AbstractSpacetime, cache)
    FiniteDiff.finite_difference_jacobian!(∂g, Skylight.metric_field(spacetime), position, cache)
    return nothing
end

function jacobian_finitediff(position, spacetime)
    ∂g = zeros(16,4)
    g = zeros(4,4)
    cache = FiniteDiff.JacobianCache(zero(position), zero(g), zero(g), Val(:central))
    finite_difference_metric_jacobian!(∂g, position, spacetime, cache)

    ∂gad = zeros(4,4,4)
    Skylight.metric_jacobian!(∂gad, position, spacetime, g)
    return reshape(∂g,4,4,4), ∂gad
end 
function benchmark_finitediff(position, spacetime)
    ∂g = zeros(16,4)
    g = zeros(4,4)
    cache = FiniteDiff.JacobianCache(zero(position), zero(g), zero(g), Val(:central))
    @btime finite_difference_metric_jacobian!($∂g, $position, $spacetime, $cache)

    ∂gad = zeros(4,4,4)
    cache = Skylight.AutoDiffChristoffelCache(spacetime)

    spacetime_metric_field = cache.spacetime_metric_field
    cfg = cache.cfg

    @btime Skylight.metric_jacobian!($∂gad, $position, $spacetime_metric_field, $g, $cfg)
    return nothing 
end 

benchmark_finitediff (generic function with 2 methods)

In [118]:
position = [0.0, 15.0, 1.0, 0.0]
spacetime = KerrSpacetimeBoyerLindquistCoordinates(1.0, 0.5)

∂g, ∂gad = jacobian_finitediff(position, spacetime)
println(all(.≈(∂g,∂gad, rtol=5e-8)))
benchmark_finitediff(position, spacetime)

true


  507.876 ns (4 allocations: 160 bytes)


  282.547 ns (4 allocations: 176 bytes)


In [112]:
FiniteDiff.default_relstep(Val(:central), Float64)

6.0554544523933395e-6